<a href="https://colab.research.google.com/github/albertlis/Plant-Disease-Recognition/blob/main/notebooks/classifier/Conv_EfficientNetB0_predictions_2_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import tensorflow as tf

drive.mount("/content/drive")
!unzip -q /content/drive/My\ Drive/PDR/Data/original.zip -d /content
!rm -r sample_data

Mounted at /content/drive


Constants

In [5]:
img_shape = (224, 224, 3)
e_net_out_shape = (7, 7, 1280)
nr_of_imgs = 49940
nr_of_val_imgs = 3862
batch_size = 64
nr_of_classes = 39
train_path = './original/train'
val_path = './original/val'

Get predictions from Convolutional part to fast train classifier

In [3]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

e_net = EfficientNetB0(include_top=False, weights="imagenet", input_shape=img_shape)

def make_conv_predictions():
    gen = ImageDataGenerator(rotation_range=45, horizontal_flip=True, vertical_flip=True, rescale=1/255)
    datagen = gen.flow_from_directory(train_path, target_size=img_shape[:2], batch_size=batch_size, class_mode='categorical')
    val_datagen = gen.flow_from_directory(val_path, target_size=img_shape[:2], batch_size=batch_size, class_mode='categorical')

    imgs = np.lib.format.open_memmap('imgs.npy', dtype='float32', mode='w+', shape=((nr_of_imgs,) + e_net_out_shape))
    labels = np.lib.format.open_memmap('labels.npy', dtype='uint8', mode='w+', shape=(nr_of_imgs, nr_of_classes))
    val_imgs = np.lib.format.open_memmap('val_imgs.npy', dtype='float32', mode='w+', shape=((nr_of_val_imgs,) + e_net_out_shape))
    val_labels = np.lib.format.open_memmap('val_labels.npy', dtype='uint8', mode='w+', shape=(nr_of_val_imgs, nr_of_classes))

    for i, (imgs_batch, labels_batch) in enumerate(datagen):
        count = i * batch_size
        line = ' '
        if not i % 20 and i != 0:
            line = '\n'
        print(f'%5d{line}' %(count), end='')
        if count > nr_of_imgs:
            break
        predictions = e_net.predict(imgs_batch)
        imgs[count : count + batch_size] = predictions
        labels[count : count + batch_size] = labels_batch
    print()

    for i, (imgs_batch, labels_batch) in enumerate(val_datagen):
        count = i * batch_size
        line = ' '
        if not i % 20 and i != 0:
            line = '\n'
        print(f'%5d{line}' %(count), end='')
        if count > nr_of_val_imgs:
            break
        predictions = e_net.predict(imgs_batch)
        val_imgs[count : count + batch_size] = predictions
        val_labels[count : count + batch_size] = labels_batch
    print()
    
make_conv_predictions()

16711680/16705208 [==============================] - 0s 0us/step
Found 49940 images belonging to 39 classes.
Found 3862 images belonging to 39 classes.
    0    16    32    48    64    80    96   112   128   144   160   176   192   208   224   240   256   272   288   304   320
  336   352   368   384   400   416   432   448   464   480   496   512   528   544   560   576   592   608   624   640
  656   672   688   704   720   736   752   768   784   800   816   832   848   864   880   896   912   928   944   960
  976   992  1008  1024  1040  1056  1072  1088  1104  1120  1136  1152  1168  1184  1200  1216  1232  1248  1264  1280
 1296  1312  1328  1344  1360  1376  1392  1408  1424  1440  1456  1472  1488  1504  1520  1536  1552  1568  1584  1600
 1616  1632  1648  1664  1680  1696  1712  1728  1744  1760  1776  1792  1808  1824  1840  1856  1872  1888  1904  1920
 1936  1952  1968  1984  2000  2016  2032  2048  2064  2080  2096  2112  2128  2144  2160  2176  2192  2208  2224  2240
 2

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, InputLayer, BatchNormalization, Dropout
import tensorflow.keras.callbacks as clb
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import backend as K 
import numpy as np
import random

def build_model():
    model = Sequential()
    
    model.add(InputLayer(input_shape=e_net_out_shape))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(659, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(39, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['acc'])
    return model

def np_array_memmap_gen(feat_path, label_path, batch_size=128, shuffle_array=True):
    while 1:
        x = np.load(feat_path, mmap_mode='r')
        y = np.load(label_path, mmap_mode='r')
        lst = [i for i in range(x.shape[0])]

        if shuffle_array:
            random.shuffle(lst)

        iters = len(lst) // batch_size + 1

        for i in range(iters):
            start = i * batch_size
            end = (i + 1) * batch_size
            yield (x[lst[start : end]], y[lst[start : end]])

callbacks = [
            clb.ReduceLROnPlateau(monitor='val_acc', factor=0.1, min_lr=1e-7, patience=2, verbose=1),
            clb.EarlyStopping(monitor='val_acc', patience=4, verbose=1),
            clb.ModelCheckpoint(monitor='val_acc', filepath='/content/drive/My Drive/PDR/Results/models/ClassifierB0_2Layers.h5',
                                save_best_only=True, verbose=1)
            ]

train_gen = np_array_memmap_gen('imgs.npy', 'labels.npy', batch_size=batch_size)
val_gen = np_array_memmap_gen('val_imgs.npy', 'val_labels.npy', batch_size=batch_size)

train_steps = nr_of_imgs // batch_size + 1
val_steps = nr_of_val_imgs // batch_size + 1

model = build_model()

history = model.fit(train_gen, epochs=50, validation_data=val_gen, callbacks=callbacks, verbose=1, 
                    steps_per_epoch=train_steps, validation_steps=val_steps)

Epoch 1/50
781/781 [==============================] - 602s 771ms/step - loss: 2.8485 - acc: 0.2791 - val_loss: 2.7997 - val_acc: 0.2885

Epoch 00001: val_acc improved from -inf to 0.28845, saving model to /content/drive/My Drive/PDR/Results/models/ClassifierB0_2Layers.h5
Epoch 2/50
781/781 [==============================] - 637s 816ms/step - loss: 1.9238 - acc: 0.4403 - val_loss: 6.1224 - val_acc: 0.1626

Epoch 00002: val_acc did not improve from 0.28845
Epoch 3/50
781/781 [==============================] - 627s 803ms/step - loss: 1.7978 - acc: 0.4738 - val_loss: 2.7276 - val_acc: 0.4039

Epoch 00003: val_acc improved from 0.28845 to 0.40394, saving model to /content/drive/My Drive/PDR/Results/models/ClassifierB0_2Layers.h5
Epoch 4/50
781/781 [==============================] - 636s 816ms/step - loss: 1.6727 - acc: 0.5076 - val_loss: 2.2529 - val_acc: 0.4068

Epoch 00004: val_acc improved from 0.40394 to 0.40678, saving model to /content/drive/My Drive/PDR/Results/models/ClassifierB0_2L